In [1]:
import psycopg2
import pandas as pd
import numpy as np
from pathlib import Path
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')

Enter database password········


# Connect to Database

In [4]:
#Connect to PgAdmin
engine = psycopg2.connect(
    database="project-data",
    user = "postgres",
    password =password,
    host = "happinessproject.cqkpnj5dcmou.us-east-1.rds.amazonaws.com",
    port = '5432'
    )

cursor = engine.cursor()


In [5]:
#Pull combined data table
query = "SELECT * from combined_data"

df = pd.read_sql(query, con=engine)

In [6]:
df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,ladder_2010,ladder_2012,ladder_2013,ladder_2014,ladder_2015,ladder_2016,ladder_2017,ladder_2018,ladder_2019,ladder_2020
0,Brazil,Latin America and Caribbean,6.330,0.043,6.415,6.245,9.577,0.882,66.601,0.804,...,6.837,7.038,7.140,6.981,6.547,6.375,6.333,6.191,6.451,6.110
1,India,South Asia,3.819,0.026,3.869,3.769,8.755,0.603,60.633,0.893,...,4.989,4.635,4.428,4.424,4.342,4.179,4.046,3.818,3.249,4.225
2,Russia,Commonwealth of Independent States,5.477,0.033,5.541,5.413,10.189,0.903,64.703,0.718,...,5.385,5.389,5.537,6.037,5.996,5.855,5.579,5.514,5.441,5.495
3,South Africa,Sub-Saharan Africa,4.956,0.060,5.074,4.839,9.403,0.860,56.904,0.749,...,4.652,4.931,3.661,4.828,4.887,4.770,4.514,4.884,5.035,4.947
4,Mexico,Latin America and Caribbean,6.317,0.053,6.420,6.213,9.859,0.831,68.597,0.862,...,6.802,6.910,7.443,6.680,6.236,6.824,6.410,6.550,6.432,5.964


# Clean master table for Machine Learning

In [7]:
#column names
list(df)

['country_name',
 'region',
 'ladder_score',
 'stand_err',
 'up_whisker',
 'low_whisker',
 'log_gdp_per_cap',
 'social_support',
 'healty_life_exp',
 'freedom',
 'generosity',
 'percept_corrupt',
 'ladder_score_distopia',
 'expby_log_gdp_per_cap',
 'expby_social_support',
 'expby_healty_life_exp',
 'expby_freedom',
 'expby_generosity',
 'expby_percept_corrupt',
 'distopia_plus_resid',
 'ac_both_sexes',
 'ac_male',
 'ac_female',
 'population_2021',
 'covid_total_cases',
 'covid_new_cases',
 'covid_total_deaths',
 'covid_new_deaths',
 'covid_total_recovered',
 'covid_new_recovered',
 'covid_active_cases',
 'covid_serious_critical',
 'covid_cases_per_mil',
 'covid_deaths_per_mil',
 'covid_total_tests',
 'covid_tests_per_mil',
 'covid_who_region',
 'population_2020',
 'land_area_skm',
 'density_skm',
 'meat_consumption',
 'median_age',
 'screen_time_avg',
 'suicide_rate',
 'ladder_2010',
 'ladder_2012',
 'ladder_2013',
 'ladder_2014',
 'ladder_2015',
 'ladder_2016',
 'ladder_2017',
 'ladde

In [11]:
#Drop columns not needed for analysis
df = df.drop(columns = ["population_2020","ladder_2010", "ladder_2012", "ladder_2013", "ladder_2014", "ladder_2015","ladder_2016","ladder_2017","ladder_2018","ladder_2019","ladder_2020"])


In [12]:
# view total null values for columns
df.isna().sum()

country_name                0
region                      1
ladder_score                0
stand_err                   0
up_whisker                  0
low_whisker                 0
log_gdp_per_cap             0
social_support              0
healty_life_exp             0
freedom                     0
generosity                  0
percept_corrupt             0
ladder_score_distopia       0
expby_log_gdp_per_cap       0
expby_social_support        0
expby_healty_life_exp       0
expby_freedom               0
expby_generosity            0
expby_percept_corrupt       0
distopia_plus_resid         0
ac_both_sexes               8
ac_male                     8
ac_female                   8
population_2021            14
covid_total_cases          14
covid_new_cases           146
covid_total_deaths         17
covid_new_deaths          147
covid_total_recovered      17
covid_new_recovered       147
covid_active_cases         17
covid_serious_critical     56
covid_cases_per_mil        14
covid_deat

In [13]:
#compare NA values to dataset row count
len(df)

149

In [15]:
#drop columns with high NA count (Na count > 75)
df = df.drop(columns=["covid_new_cases", "covid_new_deaths","covid_new_recovered","screen_time_avg"])


In [16]:
# review total null values for columns
df.isna().sum()

country_name               0
region                     1
ladder_score               0
stand_err                  0
up_whisker                 0
low_whisker                0
log_gdp_per_cap            0
social_support             0
healty_life_exp            0
freedom                    0
generosity                 0
percept_corrupt            0
ladder_score_distopia      0
expby_log_gdp_per_cap      0
expby_social_support       0
expby_healty_life_exp      0
expby_freedom              0
expby_generosity           0
expby_percept_corrupt      0
distopia_plus_resid        0
ac_both_sexes              8
ac_male                    8
ac_female                  8
population_2021           14
covid_total_cases         14
covid_total_deaths        17
covid_total_recovered     17
covid_active_cases        17
covid_serious_critical    56
covid_cases_per_mil       14
covid_deaths_per_mil      17
covid_total_tests         23
covid_tests_per_mil       23
covid_who_region          14
land_area_skm 

In [17]:
#drop all rows containing NA values
df = df.dropna()

In [18]:
# view total null values for columns
df.isna().sum()

country_name              0
region                    0
ladder_score              0
stand_err                 0
up_whisker                0
low_whisker               0
log_gdp_per_cap           0
social_support            0
healty_life_exp           0
freedom                   0
generosity                0
percept_corrupt           0
ladder_score_distopia     0
expby_log_gdp_per_cap     0
expby_social_support      0
expby_healty_life_exp     0
expby_freedom             0
expby_generosity          0
expby_percept_corrupt     0
distopia_plus_resid       0
ac_both_sexes             0
ac_male                   0
ac_female                 0
population_2021           0
covid_total_cases         0
covid_total_deaths        0
covid_total_recovered     0
covid_active_cases        0
covid_serious_critical    0
covid_cases_per_mil       0
covid_deaths_per_mil      0
covid_total_tests         0
covid_tests_per_mil       0
covid_who_region          0
land_area_skm             0
density_skm         

In [19]:
#length of final dataset for analysis 
len(df)

84

# Random Forest: Find the 10 variables that impact happiness scores the most 

In [20]:
# Create columns and target variables
columns = [
    "country_name", "region", "ladder_score", "stand_err",
    "up_whisker", "low_whisker", "log_gdp_per_cap", "social_support",
    "expby_social_support", "freedom", "generosity", "percept_corrupt",
    "ladder_score_distopia", "expby_log_gdp_per_cap", "expby_social_support", "expby_healty_life_exp",
    "expby_freedom", "expby_generosity", "expby_percept_corrupt", "distopia_plus_resid", "ac_both_sexes", "ac_male",
    "ac_female", "population_2021", "covid_total_cases", "covid_new_cases", "covid_total_deaths", "covid_total_recovered",
    "covid_new_recovered", "covid_active_cases", "covid_serious_critical", "covid_cases_per_mil", "covid_deaths_per_mil",
    "covid_total_tests", "covid_tests_per_mil", "covid_who_region", "population_2020", "land_area_skm", "land_area_skm", 
    "density_skm", "meat_consumption", "median_age", "screen_time_avg", "suicide_rate", "ladder_2010", "ladder_2012", 
    "ladder_2013", "ladder_2014", "ladder_2015", "ladder_2016", "ladder_2017", "ladder_2018", "ladder_2019", "ladder_2020"
]

target = ["ladder_score"]

In [21]:
#Convert variable type for target to non-continuous dtype
df = df.astype({'ladder_score':'int'})

df.head()

,country_name,region,ladder_score,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,...,covid_cases_per_mil,covid_deaths_per_mil,covid_total_tests,covid_tests_per_mil,covid_who_region,land_area_skm,density_skm,meat_consumption,median_age,suicide_rate
0,Brazil,Latin America and Caribbean,6,0.043,6.415,6.245,9.577,0.882,66.601,0.804,...,13716.0,464.0,13206188.0,62085.0,Americas,8515770.0,25.06,82.4,32.6,82.4
1,India,South Asia,3,0.026,3.869,3.769,8.755,0.603,60.633,0.893,...,1466.0,30.0,22149351.0,16035.0,South-EastAsia,3287263.0,454.94,5.2,28.1,5.2
2,Russia,Commonwealth of Independent States,5,0.033,5.541,5.413,10.189,0.903,64.703,0.718,...,5974.0,100.0,29716907.0,203623.0,Europe,17098242.0,8.82,51.0,39.6,51.0
3,South Africa,Sub-Saharan Africa,4,0.060,5.074,4.839,9.403,0.860,56.904,0.749,...,9063.0,162.0,3149807.0,53044.0,Africa,1219090.0,47.63,39.0,27.1,39.0
4,Mexico,Latin America and Caribbean,6,0.053,6.420,6.213,9.859,0.831,68.597,0.862,...,3585.0,391.0,1056915.0,8189.0,Americas,1964375.0,64.91,58.6,28.3,58.6


In [22]:
# Create our features
X = pd.get_dummies(df.drop(columns="ladder_score"))


# Create our target
y = df["ladder_score"]

In [23]:
X.describe()

,stand_err,up_whisker,low_whisker,log_gdp_per_cap,social_support,healty_life_exp,freedom,generosity,percept_corrupt,ladder_score_distopia,...,region_South Asia,region_Southeast Asia,region_Sub-Saharan Africa,region_Western Europe,covid_who_region_Africa,covid_who_region_Americas,covid_who_region_EasternMediterranean,covid_who_region_Europe,covid_who_region_South-EastAsia,covid_who_region_WesternPacific
count,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,8.400000e+01,...,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000,84.000000
mean,0.055190,5.742607,5.526310,9.627226,0.825512,65.973690,0.795107,-0.049690,0.753190,2.430000e+00,...,0.059524,0.035714,0.190476,0.142857,0.190476,0.214286,0.142857,0.357143,0.047619,0.047619
std,0.014841,1.025473,1.050286,1.003613,0.112456,5.847626,0.113524,0.130379,0.164949,4.467564e-15,...,0.238024,0.186691,0.395035,0.352029,0.395035,0.412790,0.352029,0.482035,0.214238,0.214238
min,0.026000,2.596000,2.449000,6.958000,0.463000,50.102000,0.382000,-0.288000,0.179000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.044500,5.031250,4.793250,9.053750,0.773750,61.999500,0.740750,-0.147500,0.720000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.053500,5.865500,5.665000,9.659500,0.850500,67.000000,0.818000,-0.075000,0.802000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.067250,6.412000,6.221000,10.372250,0.903750,69.776500,0.877500,0.037250,0.857000,2.430000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,0.092000,7.687000,7.552000,11.647000,0.954000,75.100000,0.970000,0.311000,0.939000,2.430000e+00,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [24]:
# Check the balance of our target values
y.value_counts()

5    26
4    22
6    21
7    11
3     3
2     1
Name: ladder_score, dtype: int64

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({5: 17, 4: 16, 6: 18, 7: 9, 2: 1, 3: 2})

In [26]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=128, random_state=78) 

# Creating a StandardScaler instance.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data.
predictions = rf_model.predict(X_test_scaled)

In [27]:
# Calculated the balanced accuracy score
acc_score = balanced_accuracy_score(y_test, predictions)
acc_score

0.7333333333333333

In [28]:
# Display the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[0, 1, 0, 0, 0],
       [0, 6, 0, 0, 0],
       [0, 1, 6, 2, 0],
       [0, 0, 0, 3, 0],
       [0, 0, 0, 0, 2]], dtype=int64)

In [24]:
# Print the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          3       0.00      0.00      1.00      0.00      0.00      0.00         1
          4       0.75      1.00      0.87      0.86      0.93      0.88         6
          5       1.00      0.67      1.00      0.80      0.82      0.64         9
          6       0.60      1.00      0.89      0.75      0.94      0.90         3
          7       1.00      1.00      1.00      1.00      1.00      1.00         2

avg / total       0.82      0.81      0.95      0.79      0.85      0.75        21



In [25]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.1298671005544409, 'low_whisker'),
 (0.09611501064022597, 'up_whisker'),
 (0.04398427506127815, 'expby_healty_life_exp'),
 (0.04000381293240006, 'healty_life_exp'),
 (0.0397557271930227, 'meat_consumption'),
 (0.03788461537730947, 'expby_log_gdp_per_cap'),
 (0.03541576086870717, 'log_gdp_per_cap'),
 (0.033598174674250865, 'percept_corrupt'),
 (0.03329016218732879, 'suicide_rate'),
 (0.03214294433658197, 'expby_freedom'),
 (0.0303947478803504, 'social_support'),
 (0.026609425583977213, 'expby_social_support'),
 (0.02654822479556925, 'expby_percept_corrupt'),
 (0.02644415141112979, 'freedom'),
 (0.025730504583911396, 'covid_tests_per_mil'),
 (0.024905102693394472, 'distopia_plus_resid'),
 (0.020598285760377682, 'stand_err'),
 (0.020246088264544736, 'covid_cases_per_mil'),
 (0.019749044243661574, 'median_age'),
 (0.018770097979931104, 'generosity'),
 (0.018018178909684126, 'ac_female'),
 (0.017784244750837716, 'covid_deaths_per_mil'),
 (0.016130449556759775, 'covid_total_deaths'),
 (0.